In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import os

from tensorflow.contrib.tensorboard.plugins import projector
#from tensorflow.examples.tutorials.mnist import input_data

LOG_DIR = 'clarifaisample'
NAME_TO_VISUALISE_VARIABLE = "clarifaiembedding"
#TO_EMBED_COUNT = 607  # Original code had 500, but there is a mismatch here with the sprite data for some reason

path_for_clarifai_sprites =  os.path.join(LOG_DIR,'clarifai.png')
path_for_clarifai_metadata =  os.path.join(LOG_DIR,'clarifai.tsv')

In [25]:
# load embeddings.txt as ndarray of ndarrays...
import json
embeddings_file = "../embeddings.txt"
f = open(embeddings_file)
lines = f.readlines()
f.close()
print(len(lines))
arrays = []
for ln in lines:
    ln = '['+ln.strip()+']'
    arr = json.loads(ln)
    _narr = np.array(arr)
    arrays.append(_narr)
#print(len(arrays))
_narrays = np.array(arrays)
type(_narrays), type(_narrays[0]), _narrays.shape, _narrays[0].shape
#mnist = input_data.read_data_sets("MNIST_data/", one_hot=False)
#batch_xs, batch_ys = mnist.train.next_batch(TO_EMBED_COUNT)
#batch_xs.size, batch_ys.size

213


(numpy.ndarray, numpy.ndarray, (213, 1024), (1024,))

In [26]:
embedding_var = tf.Variable(_narrays, name=NAME_TO_VISUALISE_VARIABLE)
summary_writer = tf.summary.FileWriter(LOG_DIR)


In [28]:
config = projector.ProjectorConfig()
embedding = config.embeddings.add()
embedding.tensor_name = embedding_var.name

# Specify where you find the metadata
embedding.metadata_path = path_for_clarifai_metadata #'metadata.tsv'

# Specify where you find the sprite (we will create this later)
#embedding.sprite.image_path = path_for_mnist_sprites #'mnistdigits.png'
#embedding.sprite.single_image_dim.extend([28,28])

# Say that you want to visualise the embeddings
projector.visualize_embeddings(summary_writer, config)

In [29]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver()
saver.save(sess, os.path.join(LOG_DIR, "model.ckpt"), 1)

'clarifaisample/model.ckpt-1'

In [30]:
# load the labels file
f = open("../labels.txt")
lines = f.readlines()
f.close()
lbls = []
for ln in lines:
    lbl = json.loads( ln.strip() )
    lbls.append(lbl)
print(lbls)

[0, 0, 0, 0, 1, 2, 3, 2, 4, 5, 6, 2, 7, 8, 9, 10, 11, 12, 10, 10, 11, 13, 12, 14, 10, 2, 15, 2, 16, 17, 18, 19, 20, 21, 10, 22, 22, 22, 22, 22, 22, 17, 17, 23, 22, 22, 22, 24, 24, 25, 26, 25, 23, 25, 27, 25, 27, 28, 27, 27, 28, 25, 25, 24, 23, 24, 28, 23, 24, 25, 23, 25, 27, 24, 27, 25, 27, 25, 29, 26, 30, 24, 25, 27, 25, 27, 27, 26, 27, 29, 31, 27, 25, 27, 24, 25, 27, 27, 23, 29, 25, 27, 27, 29, 27, 24, 25, 25, 25, 28, 25, 24, 25, 23, 25, 28, 28, 26, 27, 25, 25, 24, 25, 29, 28, 27, 25, 24, 25, 25, 28, 32, 25, 29, 25, 25, 25, 25, 25, 28, 25, 25, 25, 25, 25, 25, 25, 25, 23, 28, 33, 24, 25, 25, 25, 25, 25, 25, 27, 34, 25, 25, 25, 25, 25, 25, 27, 25, 28, 25, 28, 25, 27, 25, 25, 27, 25, 25, 28, 25, 25, 35, 25, 25, 25, 25, 35, 25, 25, 35, 25, 24, 27, 34, 27, 24, 24, 25, 35, 25, 34, 25, 34, 29, 25, 25, 36, 25, 27, 25, 35, 33, 25]


In [31]:
with open(path_for_clarifai_metadata,'w') as f:
    f.write("Index\tLabel\n")
    for index,label in enumerate(lbls):
        f.write("%d\t%d\n" % (index,label))

In [32]:
# Now go to a terminal and cd to where this notebook file lives and type
# >> tensorboard --logdir clarifaisample/ --port 6008

# Click on the embeddings tab and choose TSNE